# Image generator

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import numpy as np
import matplotlib.pyplot as plt
import time

from export_images import import_images
from build_model import build_generator, build_discriminator, train

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
IMG_ROWS = 100
IMG_COLS = 100
CHANNELS = 3
IMG_SHAPE = (IMG_ROWS, IMG_COLS, CHANNELS)

MODELNAME = 'jon/b128-e50'
BATCH_SIZE = 256
EPOCHS = 100

CATEGORY = "Dog"
PATH_TO_IMAGES = f"{os.getcwd()}/Images/{CATEGORY}"

## Setup Generator model and Discriminator model

In [ ]:
optimizer = Adam(0.0002, 0.5) # Learning rate, momentum


discriminator = build_discriminator(IMG_SHAPE)
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

generator = build_generator(IMG_SHAPE)
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

## Setup Combined model

In [ ]:
# Input shape
z = Input(shape=(100,))
img = generator(z)

# In the combined model, only the generator will be trained
discriminator.trainable = False 

valid = discriminator(img)


# The combined model
combined_model = Model(z, valid)
combined_model.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
# Import images, convert to float [0 1]
image_data = import_images(PATH_TO_IMAGES, (IMG_ROWS,IMG_COLS), n_images=5000) / 255
image_data = image_data[:,:,:,::-1] # BRG -> RGB

In [ ]:
train(EPOCHS, BATCH_SIZE, generator, discriminator, combined_model, image_data)

In [ ]:
# Generate image from model
noise = np.random.normal(0,1,(1,100))
gen_image = generator.predict(noise)

gen_image += 1
gen_image /= 2

#print("image: ", gen_image[0])
plt.imshow(gen_image[0])
plt.show()

In [ ]:
# Save the generator, discard the dicriminator
generator.save('./models/' + MODELNAME)